In [1]:
from datasets import load_dataset, Dataset, Features, Image, Value
from pathlib import Path
import json

from huggingface_hub import HfApi

/Users/bitan/Desktop/medgemma/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd individual/

/Users/bitan/Desktop/medgemma/individual


/Users/bitan/Desktop/medgemma/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
records = []
for line in Path("dataset.jsonl").read_text().splitlines():
    item = json.loads(line)
    records.append({
        "filename": item["filename"],                     # provided image
        "tooth": item["tooth"],
        "treatment": item["treatment"],
        "diagnosis": item["diagnosis"],
        "text": item["diagnosis"] +"; recommended treatment: " +item["treatment"] 
    })

# Define features explicitly
features = Features({
    "filename": Image(mode="RGB"),
    "tooth": Value("string"),
    "treatment": Value("string"),
    "diagnosis": Value("string"),
    "text": Value("string")
})


In [8]:
print(json.dumps(records[0], indent=2))

{
  "filename": "panoramic_img15_tooth_00001.jpg",
  "tooth": "third_molar",
  "treatment": "extraction",
  "diagnosis": "Impacted third molar",
  "text": "Impacted third molar; recommended treatment: extraction"
}


In [9]:
ds = Dataset.from_list(records, features=features)

In [10]:
print(ds.features)

{'filename': Image(mode='RGB', decode=True), 'tooth': Value('string'), 'treatment': Value('string'), 'diagnosis': Value('string'), 'text': Value('string')}


In [11]:
ds.push_to_hub(
    repo_id="justacoderwhocodes/dental_diagnosis_dataset",
    split="train",
    private=True,
    commit_message="Closeup dataset"
)


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 34.75ba/s]
Processing Files (1 / 1): 100%|██████████| 28.1MB / 28.1MB, 2.22MB/s  
New Data Upload: 100%|██████████| 28.1MB / 28.1MB, 2.22MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:16<00:00, 16.60s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/justacoderwhocodes/dental_diagnosis_dataset/commit/32253ff2f59f845da11108fd179b0113a6a1a445', commit_message='Closeup dataset', commit_description='', oid='32253ff2f59f845da11108fd179b0113a6a1a445', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/justacoderwhocodes/dental_diagnosis_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='justacoderwhocodes/dental_diagnosis_dataset'), pr_revision=None, pr_num=None)

In [12]:
check_ds = load_dataset("justacoderwhocodes/dental_diagnosis_dataset", split="train")

Generating train split: 100%|██████████| 1071/1071 [00:00<00:00, 28008.05 examples/s]


In [13]:
print(check_ds[0])

{'filename': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=448x448 at 0x11D759DE0>, 'tooth': 'third_molar', 'treatment': 'extraction', 'diagnosis': 'Impacted third molar', 'text': 'Impacted third molar; recommended treatment: extraction'}
